# ECC Pollard Rho, discrete logarithm and DES decryption
Instructions for use:

For Full Pollard Rho with 'small' examples
1. Ensure that ECC_utils.py and exampleInputRho.txt are in the same directory as this notebook
2. Run the imports cell below
3. Run the cell to read the instance from a file
3. Run the Basic Pollard Rho cell
4. Leave the example collisions commented out
5. Run the Full Pollard Rho Discrete Log solver cell
6. The output will be saved to a file called 'Full Pollard Rho Output.txt'

For running the DES Decryption example
1. Ensure that ECC_utils.py and exampleInputRho.txt are in the same directory as this notebook
2. Run the imports cell below
3. Uncomment one of the lines in the paramters cell to choose whether to use QA or QB as Q
4. Run the cell to use the parameters for ECDH
5. Run the Basic Pollard Rho cell, this will take a while to find a collision  OR  uncomment and run the example cell for the value of Q chosen, this contains c, d, c_, d_ for a valid collision
6. Run the discrete log cell to find k s.t. Q = kP
7. Run the decryption cell to decrypt the ciphertext

In [ ]:
from ECC_utils import ECC, Point, inverseModp, read_ECC_instance, write_output, write_output_full
import random
import math
from des import DesKey

## Parameters
Included are the parameters for the ECDH example given for the decryption and a function to read the parameters from a file.

The parameters are stored as global variables so that they can be accessed by any function without having to pass them as arguments

In [ ]:
# Read from a file
p, a, b, P, n, Q = read_ECC_instance('exampleInputRho.txt')

E = ECC(p, a, b)

In [ ]:
# Use parameters for ECDH
p = 20376993552394903
a = 10
b = 1
P = Point(1983, 6761152449250519)
n = 1852453970120513
QA= Point(18586784116581871, 12161036958498472)
QB= Point(18432261261031243, 11140924411855488)

# Using QA as Q
# Q = QA

# Using QB as Q
# Q = QB

E = ECC(p, a, b)

## Basic Pollard Rho method for finding a collision - identical to before

### Point transformation function

In [ ]:
def inc_mod_p(x, p):
    return (x + 1) % p
def double_mod_p(x, p):
    return (2*x) % p

def f(X, c, d):
    """
    Function for transforming an EC Point (X = c_i * P + d_i * Q) to a new, random-looking EC Point (c_i+1 * P + d_i+1 * Q)

    Parameters:
    X (Point): The point
    c (int): c s.t. X = cP + dQ
    d (int): d s.t. X = cP + dQ
    
    Returns:
    Point: The new point, X_i+1 = X_i + P or 2*X_i or X_i + Q
    int: c_i+1, after the transformation is applied
    int: c_i+1, after the transformation is applied
    """
    partition = X.x % 3
    if partition == 0:
        return E.ECPointAddition(X, P), inc_mod_p(c, n), d
    if partition == 1:
        return E.ECPointDoubling(X), double_mod_p(c, n), double_mod_p(d, n)
    if partition == 2:
        return E.ECPointAddition(X, Q), c, inc_mod_p(d, n)

def pollard_rho_ECC(E, P, Q):
    """
    Function for finding a collision

    Parameters:
    E (ECC): The curve to work on
    P (Point): The base point
    Q (Point): The point s.t. Q = lP

    Returns:
    int, int, int, int: c, d, c_, d_ s.t cP + dQ = c_P + d_Q
    """
    c = random.randint(1, p-2)
    d = random.randint(1, p-2)
    
    X = E.ECPointAddition(E.ECPointMult(c, P), E.ECPointMult(d, Q))
    X_, c_, d_ = f(X, c, d)

    while X != X_:
        X, c, d = f(X, c, d)

        X_inter, c_inter, d_inter = f(X_, c_, d_)
        X_, c_, d_ = f(X_inter, c_inter, d_inter)
    
    assert E.ECPointAddition( E.ECPointMult(c, P) , E.ECPointMult(d, Q) ) == E.ECPointAddition( E.ECPointMult(c_, P) , E.ECPointMult(d_, Q) ) == X == X_
    return c, d, c_, d_

c, d, c_, d_ = pollard_rho_ECC(E, P, Q)

# Check for case where c = c_, d=d_
if c == c_ or d == d_:
    print("Collision is identical, re-running Pollard Rho to find a new collision")
    c, d, c_, d_ = pollard_rho_ECC(E, P, Q)

## Examples

As running the above with the ECDH parameters can take a long time, I include the following example outputs

#### Example collision for ECDH (Q = QA):

* X = (14570089211650398, 11648322174886797) 
* X_ = (14570089211650398, 11648322174886797)
* c = 1312997136292347,
* d = 1257181070902744,
* c_ = 618115033144160, 
* d_ = 1308082788255376,
* k = 1682779984167835

#### And for Q = QB
* X = (19994591480866396, 581378517248716)
* X_ = (19994591480866396, 581378517248716)
* c = 74344934027957,
* d = 1839300198364846,
* c_ = 1075993571912231,
* d_ = 1000971759381135,
* k = 428971283427559

In [ ]:
# Q = QA
# c = 1312997136292347
# d = 1257181070902744
# c_ = 618115033144160 
# d_ = 1308082788255376

In [ ]:
# Q = QB
# c = 74344934027957
# d = 1839300198364846
# c_ = 1075993571912231
# d_ = 1000971759381135

# Full Pollard Rho Discrete Logarithm solver
Below I have implemented a function that takes a collision found with the pollard rho method above and uses it to solve the discrete logarithm problem to find k such that Q = kP

In [ ]:
def discrete_log(E, P, Q, c, d, c_, d_):
    '''
    Function for finding the discrete log Q = kP from a collision X = c*P + d*Q = X_ = c_*P + d_*Q

    Parameters:
    E (ECC): The curve to work on
    P (Point): The base point
    Q (Point): The point s.t. Q = kP

    Returns:
    int: k s.t. Q = kP
    '''
    gcd = math.gcd( (d_ - d), n ) # why do we do this for n, not n-1

    if gcd == 1:
        k = (c-c_) * inverseModp(d_-d, n) % n
        return int(k)
    else:
        k1 = (c-c_)/gcd * inverseModp((d_-d)/gcd, n/gcd) % n
        ks = [int(k1+(i*n/gcd)) for i in range(gcd)]
        print(f"gcd={gcd}, ks={ks}")
        for k in ks:
            if Q == E.ECPointMult(k, P):
                return k
        print(f"Failure")


k = discrete_log(E, P, Q, c, d, c_, d_)

# Output
print(f"k = {k}\nCheck that Q = kP: {E.ECPointMult(k, P) == Q}")
write_output_full(p, a, b, P, n, Q, c, d, c_, d_, k, 'Full Pollard Rho Output.txt')

## Decryption
This function takes plaintext and uses discrete_log() function to find the scalar multiples da or db used to create a key. It then recreates that key and uses it to decrypt the ciphertext C.

I have also included the scalars da and db as commented lines which can be used to test the functionality

In [ ]:
def decrypt(C, k, Q):
    # 1 find the key from db
    if Q == QA:
        key = E.ECPointMult(k, QB).x
    if Q == QB:
        key = E.ECPointMult(k, QA).x
    
    # key = 6714934996831608

    # 2 make it 56 bit binary, parity bits, convert to 8 bytes
    key = format(key, 'b')
    key = "0"*(56-len(key)) + key
    key = key[:7] + "0" + key[7:14] + "0" + key[14:21] + "0" + key[21:28] + "0" + key[28:35] + "0" + key[35:42] + "0" + key[42:49] + "0" + key[49:56] + "0"
    key = int(key, 2)
    key = key.to_bytes(8, byteorder='big')

    # 3 use des decrypt
    des_key = DesKey(key)
    C_bin = bytes.fromhex(C)
    M = des_key.decrypt(C_bin)
    M = M.decode('utf-8')
    return M

k = discrete_log(E, P, Q, c, d, c_, d_)
# k = 1682779984167835; Q = QA
# k = 428971283427559; Q = QB
C = "3da46f7b6fa82f53153908bdadcc742ac38e8691e5208aa4bf6be47240c71e75180b9d1030a00810"
decrypt(C, k, Q)

## Result

decrypt(C) = 'Joyeux Noël et Heureuse Année 2022 !\x02\x02'

The padding can be removed to reveal the intended message.